# Chapter 7 - Model Assessment and Selection

### Section 7.1 - Introduction
* The generalization performance of a learning method relates to its prediction capability on independent test data.

### Section 7.2 - Bias and Model Complexity
* Training error is the average of the loss function evaluated over the training sample.
* Test error, also referred to as generalization error, is the prediction error over an independent test sample.
* Training error is not a good estimate of the test error.
* Estimating test error is an essential task - this can be done analytically or empirically.

Two goals:
* Model selection: estimating the performance of different models in order to choose the best one.
* Model assessment: having chosen a final model, estimating its prediction error (generalization error) on new data.

The best approach for both problems is to randomly divide the dataset into three parts: 
* The training set is used to fit the models. 
* The validation set is used to estimate prediction error for model selection.
* The test set is used for assessment of the generalization error of the final chosen model. Ideally, should be kept in a “vault,” and be brought out only at the end of the data analysis.

Bias-variance tradeoff:
* As model complexity increases, typically so does prediction variance.
* As prediction variance increases, typically training error (bias) decreases.
* As prediction variance decreases, typically training error (bias) increases. This is the bias-variance tradeoff.
* Test error does not necessarily move in the same direction as train error (e.g. the more overfit you are the worse your test error can get).

In [1]:
from sklearn.datasets import load_boston
boston = load_boston()
feature_names = boston.feature_names

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.33, random_state=10)

In [3]:
# Generate bias-variance tradeoff chart for K-Neighbors regression for n_neighbors = 1 to 50. 
# Smaller n_neighbors = more model complexity.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_errors = []
train_variances = []
test_errors = []
test_variances = []
neighbor_list = list(range(1, 50))

for n_neighbors in neighbor_list:
    knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                              metric='minkowski', metric_params=None, n_jobs=-1)
    
    knn.fit(X_train, y_train)
    train_predictions = knn.predict(X_train)
    test_predictions = knn.predict(X_test)
    
    train_errors.append(mean_squared_error(y_train, train_predictions))
    test_errors.append(mean_squared_error(y_test, test_predictions))
    
    train_variances.append(train_predictions.var())
    test_variances.append(test_predictions.var())

In [4]:
import pandas as pd
import altair as alt

def _build_scatter_data(x, y, name):
    return list(zip(x, y, [name]*len(y)))

plot_data = (
    _build_scatter_data(neighbor_list, train_errors, 'Train Bias') +
    _build_scatter_data(neighbor_list, test_errors, 'Test Bias') +
    _build_scatter_data(neighbor_list, train_variances, 'Train Variance') +
    _build_scatter_data(neighbor_list, test_variances, 'Test Variance')
)
    
source = pd.DataFrame(plot_data, columns=['K', 'Value', 'Metric'])

chart = alt.Chart(source, title='Bias-Variance Tradeoff For K-Nearest Neighbors').mark_circle(size=60).encode(
    x='K',
    y='Value',
    color='Metric',
    tooltip=['K', 'Value', 'Metric']
).interactive()

chart

alt.Chart(...)

Observations:

* For K=1, as expected, we get 0 training error.
* K=1 has more test/generalization error than K=2.
* Bias and variance are inversely proportional.
* Variance increases and bias decreases as model complexity increases.

### Section 7.3 - The Bias-Variance Decomposition

Assume that assume that $Y = f(X)+\epsilon$, where $E(\epsilon) = 0$ and $Var(\epsilon) = \sigma_\epsilon^2$.

Expected prediction error of a regression fit can be analytically decomposed into several parts:

* Irreducible Error - variance of the target around its true mean $f(x_0)$, and cannot be avoided no matter how well we estimate $f(x_0)$, unless $\sigma_\epsilon^2 = 0$.
* Bias^2 - the amount by which the average of our estimate differs from the true mean.
* Variance - the expected squared deviation of $f(x_0)$ around its mean. Typically the more complex we make the model f, the lower the (squared) bias but the higher the variance.

Variance reduction techniques can often introduce *estimation bias* because they restrict the mean to be further away from the actual best fit mean. A least-squares fit, for example, will have 0 *estimation bias*.  A Ridge, while introducting estimation bias, might actually be better because of the lowered variance in predictions.  I like to think of estimation bias being the result of a specific algorithm not being an unbiased estimator of the mean.  i.e. $E(\theta) \neq \theta$.

### Section 7.4 - Optimism of the Training Error Rate
* A fitting method typically adapts to the training data, and hence the apparent or training error $\bar{err}$ will be an overly *optimistic* estimate of the generalization error $Err_T$.
* In-sample error - average error given that the new "test" observations are at the same points as the training points but with newly observed response values.
* Extra-sample error - the generalization error - test input vectors need not coincide with the training input vectors.
* Optimism of the training error is defined as the difference between the in-sample error and the training error.
* Optimism increases linearly with the number or inputs of basis functions we use, but decreases as the training sample size increases.
* Methods like AIC, BIC, and others work by estimating the optimism and adding it to the training error.
* Cross-validation and other methods like bootstrapping work by estimating the extra-sample error.
* In-sample error is usually not of direct interest but for comparison between models it can be convenient and effective.

In [5]:
# Compute AIC/BIC for different alpha's
# Degrees of freedom = those with non-zero coefficients (see section 3.5, pg 79)
from sklearn.linear_model import Lasso
import numpy as np

alphas = np.linspace(0.01, 0.20, 19)
aics = []
bics = []
degrees_of_freedom =  []
train_errors = []
train_variances = []
test_errors = []
test_variances = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, fit_intercept=True, normalize=True, precompute=False, copy_X=True, 
                  max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, 
                  selection='cyclic')

    lasso.fit(X_train, y_train)
    train_predictions = lasso.predict(X_train)
    test_predictions = lasso.predict(X_test)

    # compute AIC/BIC
    resid = y_train - train_predictions
    sse = sum(resid**2)
    k = len(lasso.coef_[lasso.coef_ != 0]) # of variables
    n = X_train.shape[0] # of observations
    aic = 2*k + n*np.log(sse/n)
    bic = k*np.log(n) + n*np.log(sse/n)
    
    aics.append(aic)
    bics.append(bic)
    degrees_of_freedom.append(k)
    
    # Compute train/test errors and variances
    train_errors.append(mean_squared_error(y_train, train_predictions))
    test_errors.append(mean_squared_error(y_test, test_predictions))
    
    train_variances.append(train_predictions.var())
    test_variances.append(test_predictions.var())

In [6]:
plot_data = (
    _build_scatter_data(alphas, train_errors, 'Train Bias') +
    _build_scatter_data(alphas, test_errors, 'Test Bias') +
    _build_scatter_data(alphas, train_variances, 'Train Variance') +
    _build_scatter_data(alphas, test_variances, 'Test Variance')
)
    
source = pd.DataFrame(plot_data, columns=['Alpha', 'Value', 'Metric'])

chart = alt.Chart(source, title='Bias-Variance Tradeoff For Lasso').mark_circle(size=60).encode(
    x='Alpha',
    y='Value',
    color='Metric',
    tooltip=['Alpha', 'Value', 'Metric']
).interactive()

chart

alt.Chart(...)

### Section 7.5 - Estimates of In-Sample Prediction Error
* Akaike information criteria (AIC) - estimate of in-sample error when a log-likelihood loss function is used. To use it for model selection, we simply choose the model with the smallest AIC. Typically this can be used to determine how complex a model should be.

In [7]:
plot_data = _build_scatter_data(alphas, bics, 'AIC')
    
source = pd.DataFrame(plot_data, columns=['Alpha', 'AIC', 'Metric'])

chart = alt.Chart(source, title='AIC For Lasso').mark_circle(size=60).encode(
    x='Alpha',
    y='AIC',
    tooltip=['Alpha', 'AIC']
).interactive()

chart

alt.Chart(...)

### Section 7.6 - The Effective Number of Parameters
* The concept of # of parameters can be generalized, for example, to cases with regularization where we are shrinking estimates.
* A linear fitting method allows us to write $\hat{y} = Sy$ where $S$ is an $NxN$ matrix depending on the input vectors $x_i$ but not on $y_i$.
* The effective # of parameters is then defined as df(S) = trace(S), the sum of the diagonal elements of $S$.

### Section 7.7 - The Bayesian Approach and BIC
* The Bayesian information criteria (BIC), like AIC, is applicable in settings where the fitting is carried out by maximization of a log-likelihood.
* Arises from the Bayesian approach to model selection - compute a posterior probability that the given model is the correct one, given the data.
* Select the model with the minimum BIC.
* Takes into account both the statistical goodness of fit and the number of parameters that have to be estimated to achieve this particular degree of fit, by imposing a penalty for increasing the number of parameters. 

In [8]:
plot_data = _build_scatter_data(alphas, bics, 'BIC')
    
source = pd.DataFrame(plot_data, columns=['Alpha', 'BIC', 'Metric'])

chart = alt.Chart(source, title='BIC For Lasso').mark_circle(size=60).encode(
    x='Alpha',
    y='BIC',
    tooltip=['Alpha', 'BIC']
).interactive()

chart

alt.Chart(...)

### Section 7.8 - Minimum Description Length
* You can use concepts from messaging coding/compression theory to derive BIC in a different manner. From a high level view it reflects the minimum amount of encoding necessary to describe your data.

### Section 7.9 - Vapnik-Chervonenkis Dimension
* In-sample error estimates need a complexity/degrees of freedom parameter, and this can not be estimated for all non-linear models using effective # of parameter methods from 7.6.  The VC theory provides such a general measure of complexity and gives a bound on optimism.
* Way of measuring the complexity of a class of functions by assessing how "wiggly" its memberes can be. For example this is important when determining how accurately a boundary function can separate classes.
* VC is defined to be the largest number of points that can be shattered by the class of functions.
* Vapnik's structural risk minimization (SRM) approach fits a nested sequence of models of increasing VC dimensions and then chooses the model with the smallest value for upper bound of optimism.

### Section 7.10 - Cross Validation
* Directly estimates the extra-sample error (the average generalization error).
* K-Fold - Split into K equal sized samples and train on K-1 of the samples and predict on the remaining sample. Repeat this K times. Combine the K estimates.
* If K = N, the number of rows, this is known as *leave-one-out* cross validation.
* Cross validation only effectively estimates the average error (generalization error).
* 5 or 10 fold validation is recommended as the right tradeoff between bias, variance, and computational constraints.
* Generalized cross-validation technique gives an approximation to leave-one-out method.
* Be careful of order of operations when doing cross validation. For example, if selecting variables based on correlation w/ response, make sure it is not done before the K-Fold validation. In general, CV must be applied to ALL modeling steps.

In [9]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import auc

In [10]:
kf = KFold(n_splits=5, shuffle=True, random_state=10)

In [11]:
X = boston.data
y = boston.target

model_objects = []

fold = 1
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    knn = KNeighborsRegressor(n_neighbors=200, weights='uniform', algorithm='auto', leaf_size=30, p=5, 
                              metric='minkowski', metric_params=None, n_jobs=-1)
    
    knn.fit(X_train, y_train)
    model_objects.append(knn)
    train_predictions = knn.predict(X_train)
    test_predictions = knn.predict(X_test)
    
    print('Fold: {} \n Train MSE: {}, Test MSE: {} \n Train Variance: {}, Test Variance: {}\n'.format(
        fold,
        mean_squared_error(y_train, train_predictions),
        mean_squared_error(y_test, test_predictions),
        train_predictions.var(),
        test_predictions.var()
    ))
    
    fold += 1

Fold: 1 
 Train MSE: 62.150713996905935, Test MSE: 94.6636065833333 
 Train Variance: 6.890965279009102, Test Variance: 5.3116149893310265

Fold: 2 
 Train MSE: 72.81067819691359, Test MSE: 59.03369190841583 
 Train Variance: 6.015169766660568, Test Variance: 6.030122304234878

Fold: 3 
 Train MSE: 67.72803781481481, Test MSE: 74.01915098762376 
 Train Variance: 7.674340321969212, Test Variance: 7.904632889177528

Fold: 4 
 Train MSE: 73.47340186172839, Test MSE: 58.45413314603961 
 Train Variance: 5.4428992285871045, Test Variance: 5.957576775855306

Fold: 5 
 Train MSE: 69.77699731604939, Test MSE: 66.16521578960396 
 Train Variance: 6.981706877177259, Test Variance: 7.537848432261543



In [12]:
predictions = []

for knn in model_objects:
    predictions.append(knn.predict(X))
    
ensembled_predictions = np.array(predictions).mean(axis = 0)

In [13]:
print('MSE: {}, Variance: {}\n'.format(
    mean_squared_error(y, ensembled_predictions),
    ensembled_predictions.var()
))

MSE: 69.31600944288537, Variance: 6.566750649521784



In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
knn = KNeighborsRegressor()
gsc = GridSearchCV(estimator=knn, param_grid = {'n_neighbors': range(1, 200)})
gsc.fit(X, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(1, 200)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [16]:
best_knn = gsc.best_estimator_

In [17]:
best_knn.n_neighbors

58

In [18]:
gsc.best_score_

-0.1630466679885652

### Section 7.11 - Bootstrap Methods
* Randomly draw datasets with replacement from the training data, each size the same as the original training set. Do this B times.
* Then, refit the model to each of the bootstrap datasets and examine the fits over the B replications.
* Approaches for estimating error:
    * Predict how well it fits the original non-bootstrapped training set and keep track of the error (average each bootstrap model error). This doesn't provide a great estimate because of its tendency to favor overfit models.
    * Keep track of predictions only from bootstrap samples not containing that prediction. That is, when predicting on the training set only use data points that do not exist in the bootstrap sample. This is like CV and called the leave-one-out bootstrap method. Its training set bias will behave like two-fold CV. An unbiased version is 0.368 * training error + 0.632 * leave_one_out_error. This formula has a complicated derivation.
* For many adaptive, non-linear methods, estimators like AIC are impractical, leaving us with CV and bootstrap.
* **With methods like fitting trees, cross-validation and bootstrap can understimate the true error by 10%, because the search for best tree is strongly affected by the validation set. In these situations only a separate test set will provide an unbiased estimate of the test error.**


In [19]:
from sklearn.utils import resample
import numpy as np

In [20]:
B = 10
errors = []

for i in range(B):
    X_bootstrap, y_bootstrap = resample(X, y)
    
    knn = KNeighborsRegressor(n_neighbors=50, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                          metric='minkowski', metric_params=None, n_jobs=-1)
    
    knn.fit(X_bootstrap, y_bootstrap)
    predictions = knn.predict(X)
    e = mean_squared_error(y, predictions)
    errors.append(e)
    
    print('Sample: {} \n MSE: {}, Variance: {}\n'.format(
        i+1,
        e,
        predictions.var()
    ))
    
print('Avg. MSE: {}'.format(np.array(errors).mean()))

Sample: 1 
 MSE: 58.46088671936759, Variance: 18.71067635307535

Sample: 2 
 MSE: 57.20755137549407, Variance: 19.748870403115188

Sample: 3 
 MSE: 56.701366, Variance: 16.487710413301258

Sample: 4 
 MSE: 56.85304129644268, Variance: 22.339110184505305

Sample: 5 
 MSE: 60.155956079051386, Variance: 24.089078261931913

Sample: 6 
 MSE: 57.67518415019763, Variance: 26.058396206486584

Sample: 7 
 MSE: 57.41876413438735, Variance: 20.010678020231527

Sample: 8 
 MSE: 57.82781815019762, Variance: 23.491621240013124

Sample: 9 
 MSE: 59.29024582608696, Variance: 21.558376858894846

Sample: 10 
 MSE: 57.91658765217392, Variance: 17.722285617397553

Avg. MSE: 57.950740138339924


In [21]:
# TO DO - implement error estimate for samples not in bootstrap (leave-one-out bootstrap)

### Section 7.12 - Conditional or Expected Test Error
* K-Fold validation does not estimate training error well.
* K-Fold validation DOES estimate expected error (generalization error) well and in an unbiased manner, even though the variation in test error for different training sets (folds) is substantial.
* Estimation of test error for a particular training set is not easy in general, given just the data from that same training set. Instead, CV and related methods may provide reasonable estimates of the *expected* error.